In [ ]:
#Group Project
#AAI - 530
#Smart Agriculture Monitoring & Decision Support

#Gaius Thomas
#Jasper A. Dolar

In [ ]:
#CELL 0 - Environment and Reproducibility Setup

import os
import glob
import warnings

import numpy as np
import pandas as pd

#for small n = 99 
from sklearn.model_selection import LeaveOneOut 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

#ignore output warnings
warnings.filterwarnings("ignore")

#reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

#control for display size
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)


In [4]:
#CELL 1 - Define Project Paths & Verify Raw data Files

#print current working dir
print("Current working directory:", os.getcwd())

RAW_DIR = os.path.join("..", "data", "raw")

#we'll store cleaned/merged outputs in ../data/processed
PROCESSED_DIR = os.path.join("..", "data", "processed")

#create processed directory if it doesn't exist
os.makedirs(PROCESSED_DIR, exist_ok=True)
print("\nRAW_DIR (absolute):", os.path.abspath(RAW_DIR))
print("PROCESSED_DIR (absolute):", os.path.abspath(PROCESSED_DIR))

#collect all Excel files from raw dir
xlsx_files = sorted (glob.glob(os.path.join(RAW_DIR, "*.xlsx")))
print(f"\nFound {len(xlsx_files)} Excel files(s) in data/raw:")
for f in xlsx_files:
    print(" -", os.path.basename(f))

#if no files found:
if len(xlsx_files) == 0:
    print("No .xlsx files found.")


Current working directory: /Users/jd/Documents/USD-MSAAI/IoTAgricultureProject/code

RAW_DIR (absolute): /Users/jd/Documents/USD-MSAAI/IoTAgricultureProject/data/raw
PROCESSED_DIR (absolute): /Users/jd/Documents/USD-MSAAI/IoTAgricultureProject/data/processed

Found 8 Excel files(s) in data/raw:
 - Environment Humidity (1).xlsx
 - Environment Light Intensity (1).xlsx
 - Environment Temperature (1).xlsx
 - Soil Moisture (1).xlsx
 - Soil Temperature.xlsx
 - Soil pH.xlsx
 - Solar Panel Battery Voltage.xlsx
 - Water TDS.xlsx


In [6]:
#CELL 2 - Load Raw Excel Files & Inspect Structure

#create dictionary to store each sensor DataFrame keyed by 
#its filename
sensor_dfs={}

#loop through each Excel file path found in Cell 1
for file_path in xlsx_files:
    #extract only the file name 
    file_name = os.path.basename(file_path)

    #read the Excel file into pandas dataframe
    df = pd.read_excel(file_path)

    #store the dataframe in the dictionary so we can clean or merge later
    sensor_dfs[file_name] = df

    #print basic information
    print(f"\n Loaded File: {file_name}")
    print("Shape (rows, cols):", df.shape)
    print("Column names:", list(df.columns))

    #display first 5 rows to visually confirm
    #timestamp/value columns
    display(df.head())


 Loaded File: Environment Humidity (1).xlsx
Shape (rows, cols): (101, 3)
Column names: ['Environment Humidity', 'Unnamed: 1', 'Unnamed: 2']


,Environment Humidity,Unnamed: 1,Unnamed: 2
0,Date & Time Created,Entry_id,Measurement Unit(%)
1,2024-09-27 12:58:10 +0530,2245,95
2,2024-09-27 12:58:28 +0530,2246,95
3,2024-09-27 12:59:03 +0530,2247,95
4,2024-09-27 12:59:22 +0530,2248,95



 Loaded File: Environment Light Intensity (1).xlsx
Shape (rows, cols): (101, 3)
Column names: ['Environment Light Intensity', 'Unnamed: 1', 'Unnamed: 2']


,Environment Light Intensity,Unnamed: 1,Unnamed: 2
0,Date & Time of Creation,Entry_id,Mesaurement Unit(Lux)
1,2024-09-27 12:58:10 +0530,2245,50.83
2,2024-09-27 12:58:28 +0530,2246,36.67
3,2024-09-27 12:59:03 +0530,2247,39.17
4,2024-09-27 12:59:22 +0530,2248,39.17



 Loaded File: Environment Temperature (1).xlsx
Shape (rows, cols): (101, 3)
Column names: ['Environment Temperature', 'Unnamed: 1', 'Unnamed: 2']


...

,Soil pH,Unnamed: 1,Unnamed: 2
0,Date & Time Created,Entry_id,Measurement Unit (pH)
1,2024-09-27 12:58:10 +0530,2245,6.1
2,2024-09-27 12:58:28 +0530,2246,6.1
3,2024-09-27 12:59:03 +0530,2247,6.1
4,2024-09-27 12:59:22 +0530,2248,6.1



 Loaded File: Solar Panel Battery Voltage.xlsx
Shape (rows, cols): (101, 3)
Column names: ['Solar Panel Battery Voltage', 'Unnamed: 1', 'Unnamed: 2']


,Solar Panel Battery Voltage,Unnamed: 1,Unnamed: 2
0,Date & Time Created,Entry_id,Measurement Unit(Volts)
1,2024-09-27 12:58:10 +0530,2245,3.55
2,2024-09-27 12:58:28 +0530,2246,3.552
3,2024-09-27 12:59:03 +0530,2247,3.554
4,2024-09-27 12:59:22 +0530,2248,3.556



 Loaded File: Water TDS.xlsx
Shape (rows, cols): (101, 3)
Column names: ['Water TDS', 'Unnamed: 1', 'Unnamed: 2']


,Water TDS,Unnamed: 1,Unnamed: 2
0,Date & Time Created,Entry_id,Measurement Unit(mg/L)
1,2024-09-27 12:58:10 +0530,2245,130.68
2,2024-09-27 12:58:28 +0530,2246,128.89
3,2024-09-27 12:59:03 +0530,2247,128.89
4,2024-09-27 12:59:22 +0530,2248,155.42


In [7]:
#CELL 3 - Clean, Standardize, and Prepare Each Sensor

#This will remove metadata rows, standardize column names, 
#and parse time stamps

#dictionary to store cleaned sensor DataFrames
cleaned_sensor_dfs = {}

#loop through each raw sensor DataFrame
for file_name, df in sensor_dfs.items():

    #make a copy so we do not overwrite raw data
    df_clean = df.copy()

    #drop the first row (metadata row, not actual sensor data)
    df_clean = df_clean.iloc[1:].reset_index(drop=True)

    #rename columns to a consistent schema
    df_clean.columns = ["timestamp", "entry_id", "value"]

    #convert timestamp column to pandas datatime 
    df_clean["timestamp"] = pd.to_datetime(df_clean["timestamp"], errors="coerce")

    #convert sensor values to numeric 
    df_clean["value"] = pd.to_numeric(df_clean["value"], errors="coerce")

    #sort by timestamp to ensure correct temporal order
    df_clean = df_clean.sort_values("timestamp").reset_index(drop=True)

    #create a clean sensor name from the filename
    sensor_name = (
        file_name.replace(".xlsx", "")
        .replace(" (1)", "")
        .lower()
        .replace(" ", "_")
    )

    #rename value column to sensor-specific name
    df_clean = df_clean.rename(columns={"value": sensor_name})

    #store cleaned DataFrame
    cleaned_sensor_dfs[sensor_name] = df_clean

    #print confirmation to check
    print(f"Cleaned sensor: {sensor_name}")
    print("Shape:", df_clean.shape)
    display(df_clean.head())


Cleaned sensor: environment_humidity
Shape: (100, 3)


,timestamp,entry_id,environment_humidity
0,2024-09-27 12:58:10+05:30,2245,95
1,2024-09-27 12:58:28+05:30,2246,95
2,2024-09-27 12:59:03+05:30,2247,95
3,2024-09-27 12:59:22+05:30,2248,95
4,2024-09-27 12:59:45+05:30,2249,95


Cleaned sensor: environment_light_intensity
Shape: (100, 3)


,timestamp,entry_id,environment_light_intensity
0,2024-09-27 12:58:10+05:30,2245,50.83
1,2024-09-27 12:58:28+05:30,2246,36.67
2,2024-09-27 12:59:03+05:30,2247,39.17
3,2024-09-27 12:59:22+05:30,2248,39.17
4,2024-09-27 12:59:45+05:30,2249,30.83


Cleaned sensor: environment_temperature
Shape: (100, 3)


...

,timestamp,entry_id,soil_ph
0,2024-09-27 12:58:10+05:30,2245,6.1
1,2024-09-27 12:58:28+05:30,2246,6.1
2,2024-09-27 12:59:03+05:30,2247,6.1
3,2024-09-27 12:59:22+05:30,2248,6.1
4,2024-09-27 12:59:45+05:30,2249,6.1


Cleaned sensor: solar_panel_battery_voltage
Shape: (100, 3)


,timestamp,entry_id,solar_panel_battery_voltage
0,2024-09-27 12:58:10+05:30,2245,3.550
1,2024-09-27 12:58:28+05:30,2246,3.552
2,2024-09-27 12:59:03+05:30,2247,3.554
3,2024-09-27 12:59:22+05:30,2248,3.556
4,2024-09-27 12:59:45+05:30,2249,3.557


Cleaned sensor: water_tds
Shape: (100, 3)


,timestamp,entry_id,water_tds
0,2024-09-27 12:58:10+05:30,2245,130.68
1,2024-09-27 12:58:28+05:30,2246,128.89
2,2024-09-27 12:59:03+05:30,2247,128.89
3,2024-09-27 12:59:22+05:30,2248,155.42
4,2024-09-27 12:59:45+05:30,2249,130.68


In [8]:
#CELL 4 - Merge ALl Sensors into One Master Dataset
#Combine all cleaned sensor DataFrames into a single time-aligned dataset

#get lst of cleaned sensor names
sensor_keys = list(cleaned_sensor_dfs.keys())

#initialize master DataFrame using the firste sensor
master_df = cleaned_sensor_dfs[sensor_keys[0]].copy()

#merge remaning sensors one-by-one on timestamp
for sensor in sensor_keys[1:]:
    #drop entry_id from subsequence DataFrames to avoid duplicate columns
    df_to_merge = cleaned_sensor_dfs[sensor].drop(columns=["entry_id"])

    #outer merge preserves all timestamps across sensors
    master_df = pd.merge(
        master_df, 
        df_to_merge,
        on="timestamp",
        how="outer"
    )

#sort final dataset by timestamp and reset index
master_df = master_df.sort_values("timestamp").reset_index(drop=True)

#display merged dataset structure
print("Master Dataset Created")
print("Shape (rows, columns):", master_df.shape)
print("Columns:", list(master_df.columns))

display(master_df.head())

#check missing values per column (expected in multi-sensor IoT data)
print("\nMissing values per column: ")
print(master_df.isna().sum())

#save merged dataset for ML models and Tableau dashboard
output_path = os.path.join(PROCESSED_DIR, "merged_sensor_data.csv")
master_df.to_csv(output_path, index=False)

print(f"\n Saved merged dataset to: {output_path}")

Master Dataset Created
Shape (rows, columns): (101, 10)
Columns: ['timestamp', 'entry_id', 'environment_humidity', 'environment_light_intensity', 'environment_temperature', 'soil_moisture', 'soil_temperature', 'soil_ph', 'solar_panel_battery_voltage', 'water_tds']


,timestamp,entry_id,environment_humidity,environment_light_intensity,environment_temperature,soil_moisture,soil_temperature,soil_ph,solar_panel_battery_voltage,water_tds
0,2024-09-27 12:58:10+05:30,2245,95.0,50.83,26.2,61.88,18.1,6.1,3.550,130.68
1,2024-09-27 12:58:28+05:30,2246,95.0,36.67,26.2,61.58,18.1,6.1,3.552,128.89
2,2024-09-27 12:59:03+05:30,2247,95.0,39.17,26.2,61.88,18.1,6.1,3.554,128.89
3,2024-09-27 12:59:22+05:30,2248,95.0,39.17,26.2,59.24,18.1,6.1,3.556,155.42
4,2024-09-27 12:59:45+05:30,2249,95.0,30.83,26.2,31.48,18.1,6.1,3.557,130.68



Missing values per column: 
timestamp                      0
entry_id                       1
environment_humidity           1
environment_light_intensity    1
environment_temperature        1
soil_moisture                  1
soil_temperature               1
soil_ph                        1
solar_panel_battery_voltage    1
water_tds                      1
dtype: int64

 Saved merged dataset to: ../data/processed/merged_sensor_data.csv
